In [1]:
import numpy as np
import pytorch_model_summary
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import FashionMNIST

from functions.callbacks import MlflowLogger, ModelCheckPoint
from model.cnn_model import Net

/home/ec2-user/anaconda3/envs/env1/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(f"cuda : {torch.cuda.is_available()}")
device = torch.device("cuda")
lr = 1e-3
batch_size = 16

train_dataset = FashionMNIST(
    root='./dataset/FashionMnist', train=True, download=True,
    transform=lambda x: torch.tensor((x - np.mean(x)) / np.std(x), device=device)
)
test_dataset = FashionMNIST(
    root='./dataset/FashionMnist', train=False, download=True,
    transform=lambda x: torch.tensor((x - np.mean(x)) / np.std(x), device=device)
)

train_iterator = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_iterator = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

model = Net(device=device)
model.double()

cuda : True


Net(
  (conv2d): Conv2d(1, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (dropout): Dropout(p=0.1, inplace=False)
  (relu): ReLU()
  (softmax): Softmax(dim=1)
  (linear): Linear(in_features=588, out_features=10, bias=True)
  (linear2): Linear(in_features=10, out_features=10, bias=True)
)

In [3]:
model_input = next(iter(train_iterator))
print(
    pytorch_model_summary.summary(
        model, model_input[0], show_input=True
    )
)

-----------------------------------------------------------------------
      Layer (type)         Input Shape         Param #     Tr. Param #
          Conv2d-1     [16, 1, 28, 28]              30              30
            ReLU-2     [16, 3, 28, 28]               0               0
         Dropout-3           [16, 588]               0               0
          Linear-4           [16, 588]           5,890           5,890
          Linear-5            [16, 10]             110             110
         Softmax-6            [16, 10]               0               0
Total params: 6,030
Trainable params: 6,030
Non-trainable params: 0
-----------------------------------------------------------------------


-----

In [4]:
cross_entropy_loss = torch.nn.CrossEntropyLoss().to(model.device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
callbacks = []

In [5]:
model.fit(
    10, train_iterator, test_iterator, 
    loss_func=cross_entropy_loss, optimizer=optimizer, 
    callback=callbacks
)

 train 001/10 epoch [■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■] 100.00% Done 20.59 sec loss : 1.722 acc : 0.743  val_loss : 1.613  val_acc : 0.851
 train 002/10 epoch [■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■] 100.00% Done 20.61 sec loss : 1.615 acc : 0.847  val_loss : 1.598  val_acc : 0.864
 train 003/10 epoch [■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■] 100.00% Done 20.68 sec loss : 1.604 acc : 0.857  val_loss : 1.601  val_acc : 0.862
 train 004/10 epoch [■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■] 100.00% Done 20.68 sec loss : 1.599 acc : 0.863  val_loss : 1.588  val_acc : 0.873
 train 005/10 epoch [■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■] 100.00% Done 20.55 sec loss : 1.594 acc : 0.867  val_loss : 1.580  val_acc : 0.881
 train 006/10 epoch [■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■] 100.00% Done 20.52 sec loss : 1.592 acc : 0.869  val_loss : 1.587  val_acc : 0.874
 train 007/10 epoch [■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■] 100.00% Done 20.58 sec loss : 1.591 acc : 0.870  val_loss : 1.578  val_acc : 0.883
 train 008/10 epoch [■■■■■■■■■■■■■■■■■■■■■■■■■■■

-----

In [7]:
def multiplied_mse(output, label):
    bs = len(label)
    idx = torch.tensor(range(bs))
    one_hot_label = torch.zeros([16, 10], device=device)
    one_hot_label[idx, label] = 1
    return torch.mean((output - one_hot_label)**2) * 20

In [5]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
callbacks = []

In [8]:
model.fit(
    10, train_iterator, test_iterator, 
    loss_func=multiplied_mse, optimizer=optimizer, 
    callback=callbacks
)

 train 001/10 epoch [■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■] 100.00% Done 21.82 sec loss : 0.572 acc : 0.803  val_loss : 0.404  val_acc : 0.862
 train 002/10 epoch [■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■] 100.00% Done 21.83 sec loss : 0.414 acc : 0.859  val_loss : 0.365  val_acc : 0.877
 train 003/10 epoch [■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■] 100.00% Done 21.93 sec loss : 0.390 acc : 0.867  val_loss : 0.355  val_acc : 0.879
 train 004/10 epoch [■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■] 100.00% Done 21.95 sec loss : 0.377 acc : 0.872  val_loss : 0.339  val_acc : 0.885
 train 005/10 epoch [■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■] 100.00% Done 21.97 sec loss : 0.366 acc : 0.875  val_loss : 0.330  val_acc : 0.888
 train 006/10 epoch [■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■] 100.00% Done 21.93 sec loss : 0.361 acc : 0.877  val_loss : 0.318  val_acc : 0.893
 train 007/10 epoch [■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■] 100.00% Done 21.90 sec loss : 0.358 acc : 0.879  val_loss : 0.318  val_acc : 0.894
 train 008/10 epoch [■■■■■■■■■■■■■■■■■■■■■■■■■■■

-----

In [11]:
def fake_loss(output, label):
    return torch.tensor(0.0, device=device, requires_grad=True)

In [12]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
callbacks = []

In [14]:
model.fit(
    2, train_iterator, test_iterator, 
    loss_func=fake_loss, optimizer=optimizer, 
    callback=callbacks
)

 train 001/2 epoch [■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■] 100.00% Done 14.53 sec loss : 0.000 acc : 0.089  val_loss : 0.000  val_acc : 0.089
 train 002/2 epoch [■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■] 100.00% Done 14.56 sec loss : 0.000 acc : 0.088  val_loss : 0.000  val_acc : 0.089
